In [ ]:
# initial setup for any notebook

%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/yarcoh/projects/thesis-code4") # go to parent dir

In [ ]:
import numpy as np
from ModulationPy import ModulationPy
from commpy import rrcosfilter
from matplotlib import pyplot as plt
from IPython.display import display, Markdown


from src.optics.split_step_fourier import SplitStepFourier
from src.general_methods.visualizer import Visualizer
from src.general_methods.signal_processing import SP
from src.optics.myFNFTpy.FNFTpy import nsev_inverse_xi_wrapper, nsev_inverse, nsev

In [ ]:
# ------------------ params ------------------
# System Configuration:
W = 50 / 1000           # Total bandwidth, estimated [THz]
Nspans = 12             # The number of spans
La = 80                 # Transmission span [km]
M_QAM = 16              # QAM order (2,4,16,64,256)

# Modulation and Coding:
Nos = 16                # Oversampling factor (must be even)
eta = 2                 # spectral efficiency penalty factor (1,4]
# eta = 1.1             # spectral efficiency penalty factor (1,4]
mu = 0.15               # Dimensionless power scaling factor (RRC)
# mu = 10              # Dimensionless power scaling factor (RRC)
bet = 0.2               # roll-off factor
with_ssf = True         # whether to use SSF or not
with_noise = False       # whether to add noise or not

# Fiber and Dispersion:
alphadB = 0.2           # Power loss db/km
beta2 = -21             # ps^2/km
gamma = 1.27            # Nonlinear coefficient in [1/km*W]

# direct calculations
L = La*Nspans  # km
alpha = alphadB*np.log(10)/10       # Power loss [1/km]


# calculations
# The required guard band [ps] + 50% increase
T_guardband = 1.5*np.pi*W*abs(beta2)*L
N_sc_raw = W*T_guardband / (eta-1)                   # The required number of subcarriers
N_sc = int(2**np.ceil(np.log2(N_sc_raw)))   # rounded to nearest factor of two
T0 = N_sc/W                                 # Useful symbol duration, normalization time [ps]
Tb = T0*eta                                 # Burst width
lumped = False
if lumped:
    G = np.exp(alpha*La)                        # The power gain factor
    if G > 1:
        gamma_eff = gamma*(G-1)/(G*np.log(G))
else:
    G = 0
    alpha = 0
    gamma_eff = gamma

display(Markdown(f"""
## params:
#### System Configuration:
* $ W = {W} [THz] $
* $ N_{{spans}} = {Nspans} $
* $ L_a = {La} [km] \\longrightarrow L = L_a N_{{spans}} = {La} \\cdot {Nspans} = {L} $
* $ M_{{QAM}} = {M_QAM} $

#### Modulation and Coding:
* $ N_{{os}} = {Nos} $
* $ \\eta = {eta} $
* $ \\mu = {mu} $
* $ \\beta = {bet} $

#### Fiber and Dispersion:
* $ \\alpha = {alphadB} [dB/km] = {alpha:.3f} [1/km]$
* $ \\beta_2 = {beta2} [ps^2/km] $
* $ \\gamma = {gamma} [1/(kmW)] $                 

#### calculations:
* $ T_G=150\% \\cdot \\pi W |\\beta_2|L =1.5 \\cdot \\pi \\cdot {W} \\cdot  |{beta2}| \\cdot {L} = {int(T_guardband/np.pi)}\\pi = {T_guardband:.2f} $
* $ N_{{sc}} = \\lceil \\frac{{W T_G}}{{\\eta-1}} \\rceil= \\lceil \\frac{{ {W}*{T_guardband//np.pi}\\pi }}{{ {eta}-1 }} \\rceil = \\lceil {N_sc_raw/np.pi} \\pi \\rceil= {N_sc} $
* $ T_0 = \\frac{{N_{{sc}}}}{{W}} = \\frac{{ {N_sc} }}{{ {W} }} = {int(T0)} [ps]$
* $ T_b = \\eta T_0 = {eta} \\cdot {int(T0)} = {int(Tb)} [ps] = {Tb/1e3} [ns]$
* if lumped:
    * $ G = e^{{\\alpha L_a}} = e^{{ {alpha:.3f} \\cdot {La} }} = {G:.3f}$
        * $ G>1 \\rightarrow $
            * $ \\gamma_{{eff}} = \\frac{{\\gamma(G-1)}}{{G \\ln G}} = \\frac{{ {gamma}({G:.3f}-1)}}{{ {G:.3f} \\ln {G:.3f}}} = {gamma_eff:.3f}$ [ 1/(kmW) ]
* else:
    * $ \\gamma_{{eff}} = \\gamma = {gamma_eff} $
    * $ \\alpha = {alpha} $
"""))


In [ ]:
# normalized units
Tn = T0/(np.pi*(1+bet))             # [ps]
Zn = Tn**2/abs(beta2)               # [km]
Pn = 1/(gamma_eff*Zn)               # [W]

dz = 0.2                            # Z-step, [km] - initial step estimate
Nsps = np.ceil(La/dz)               # Required number of steps per span


display(Markdown(f"""
## Normalized units:
* $T_n = \\frac{{T_0}}{{\\pi(1+\\beta)}} = \\frac{{ {T0} }}{{ \\pi (1+ {bet}) }} = {Tn:.2f}$ $[ps] $
* $Z_n = \\frac{{T_n^2}}{{|\\beta_2|}} = \\frac{{ {Tn:.2f}^2}}{{| {beta2} |}} = {Zn:.2f}$ $[km] $
* $P_n = \\frac{{1}}{{\\gamma_{{eff}} Z_n}} = \\frac{{1}}{{ {gamma_eff:.2f} \\cdot {Zn:.2e} }} = {Pn:.2e}$ $[W] $

#### physical channel params:
* $ dz = {dz} [km] $
* $ N_{{sps}} = \\lceil \\frac{{L_a}}{{dz}} \\rceil = \\lceil \\frac{{ {La} }}{{ {dz} }} \\rceil = {Nsps} $
"""))

In [ ]:
# axes
Ns = N_sc*Nos                           # The number of meaningful points
Nnft = int(4*2**(np.ceil(np.log2(Ns)))) # The number of points for NFT - round up to a power of 2
Tnft = np.pi*Nos*Tn                     # The NFT base
dt = Tnft/Nnft                          # The time step in ps
Nb = Tb/dt                              # The number of points in each burst
Nb = 2*round(Nb/2)                      # Make sure it is even

T1 = dt*(-Nnft/2)/Tn
T2 = dt*(Nnft/2-1)/Tn
_, XI = nsev_inverse_xi_wrapper(Nnft,T1,T2,Nnft,display_c_msg=True)
xi = np.arange(-Ns/2, Ns/2)/Nos             # Array of upsampled nonlinear frequencies
xi_padded = np.linspace(XI[0],XI[1],Nnft)   # Array of padded nonlinear frequencies (for NFT)
t = np.arange(-Nb/2, Nb/2)*dt
t_padded = np.linspace(T1,T2, Nnft)



display(Markdown(f"""
## axes specs
* $ N_s = N_{{sc}} N_{{os}} = {N_sc} \\cdot {Nos} = {Ns} $
* $ N_{{NFT}} = 4 \\cdot 2^{{\\lceil \\log_2(N_s) \\rceil}} = {Nnft} $
* $ T_{{NFT}} = \\pi \\cdot N_{{os}} \\cdot T_n = \\pi \\cdot {Nos} \\cdot {Tn:.2f} = {Tnft:.2f} $
* $ \\Delta t = \\frac{{T_{{NFT}}}}{{N_{{NFT}}}} = {dt:.2f} $
* $ \\Delta \\xi = {xi[1]-xi[0]} $
* $ N_b = \\lceil \\frac{{T_b}}{{\\Delta t}} \\rceil = \\lceil \\frac{{ {Tb} }}{{ {dt:.2f} }} \\rceil = {Nb} $
* $ T_1 = \\frac{{\\Delta t}}{{T_n}}  \\cdot (-N_{{NFT}}/2)= {T1:.2f} $
* $ T_2 = \\frac{{\\Delta t}}{{T_n}} \\cdot (N_{{NFT}}/2-1) = {T2:.2f} $

#### axes:
* $ \\xi \\in [{int(xi[0])}, {int(xi[-1])}], 
    \qquad \qquad \qquad \qquad \qquad \qquad N_{{\\xi}} = N_s = {len(xi)}$ 
* $ \\xi_{{padded}} \\in [X_1,X_2] = [{int(xi_padded[0])}, {int(xi_padded[-1])}], 
    \qquad \qquad \qquad N_\\xi = N_{{NFT}} = {Nnft} $

* $ t \\in [-\\frac{{N_b}}{{2}}, \\frac{{N_b}}{{2}}] \\cdot \\Delta t 
    = [{-Nb/2*dt:.2f}, {Nb/2*dt:.2f}] 
    \qquad N_{{t}} = N_b = {Nb}$
* $ t_{{pad}} \\in [T_1,T_2] = [{T1:.2f},{T2:.2f}] 
    \qquad \qquad \qquad N_{{t_{{pad}}}} = N_{{NFT}} = {Nnft}$

"""))



In [ ]:
## 0) input message
message_s_int = np.random.randint(0, M_QAM, size=N_sc)
sps = int(np.log2(M_QAM))
message_s_bin = SP.unpackbits(message_s_int, sps)


msg_txt = Visualizer.vec2str(message_s_int, n_start=4, n_end=2)
bin_txt = np.reshape(message_s_bin, (-1, sps))

display(Markdown(f"""

## 0) input message
### int message 
we will generate message with Nsc [{N_sc}] words, each word has M [{M_QAM}] bits
* $ s_{{int}} \in \\mathbb Z_M ^{{N_{{sc}}}} $
* $N_{{sc}}={len(message_s_int)}$
* $M={M_QAM}$
* $ x \\in [0, {M_QAM}) \; \\forall x \\in s_{{int}} $ 

$s_{{int}} = {msg_txt}$
### binary message
now we will convert it to binary:
* $ s_b \in \\mathbb B_2 ^{{N_s}} $
* $ N_{{binary}} = N_{{sc}} \\cdot log2(M) = {len(message_s_bin)}$ 

$ s_b = $
"""))
print(bin_txt)


In [ ]:
## 1) Modulation
modem = ModulationPy.QAMModem(M_QAM, soft_decision=False)
c_in = modem.modulate(message_s_bin)

assert len(c_in) % N_sc == 0, 'number of symbols must be multiple of N_sc'

# plot

display(Markdown(f"""
## 1) Modulation
$ c[\\cdot] = f_{{MOD}}(s_b) = [c_0, c_1, ... , c_{{N_sc-1}}] $

s.t:
* $ c_i = a_i + jb_i $
* num of symbols: $ N_{{sc}} = {len(c_in)} $
"""))
Visualizer.plot_constellation_map_with_points(c_in, M_QAM, 'clean before channel')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4))
Visualizer.my_plot(np.real(c_in[0:30]), function='stem', name='i (real)', ax=ax1, hold=1)
Visualizer.my_plot(np.imag(c_in[0:30]), function='stem', name='q (imag)', ax=ax2)


In [ ]:
## 2) over sampling
c_in1 = np.zeros(Ns, dtype=np.complex64)
c_in1[::Nos] = c_in

unitless_axis = np.arange(0, len(c_in1))/Nos

# plot
display(Markdown(f"""
## 2) over sampling

$ c_1 = [c_0, \\underbrace{{[0, 0, ..., 0]}}_{{N_{{os}}-1}}, c_1, \\underbrace{{[0, 0, ..., 0]}}_{{N_{{os}}-1}}, ..., c_{{\\hat N}}, \\underbrace{{[0, 0, ..., 0]}}_{{N_{{os}}-1}}] $

s.t:
* $ c_1 \\in \\mathbb C^{{N_s}} $
* $ N_s = N_{{sc}} \\cdot N_{{os}} = {N_sc} \\cdot {Nos} = {Ns} $
"""))

Visualizer.my_plot(unitless_axis[0:50], np.real(c_in1[0:50]), name='zero padded - $ real\{c_1 \} $', function='stem')

In [ ]:
# 3) Spectral shaping
Ts = 1  # symbol period
fs = Nos/Ts


h_ind, psi_xi = rrcosfilter(N=Ns, alpha=bet, Ts=Ts, Fs=fs)
psi_t = np.fft.fft(psi_xi)  # can be saved for efficiency

# u_in = np.convolve(h_rrc, c_in_hat)  # Waveform with PSF
u_in = np.fft.ifft(np.fft.fft(c_in1) * psi_t)

assert len(u_in) == Ns, f'len(u_in) = {len(u_in)} != {Ns} = Ns'

# xi axis should be 1/ over sampling
# xi_axis = np.arange(-len(u_in)//2, len(u_in)//2)/Nos

# plot
display(Markdown(f"""
## 3) Spectral shaping
* $ u(\\xi) = \\hat c[\\cdot] \\ast \\psi(\\xi) = \\mathcal F^{{-1}}\{{C \\cdot \\Psi \}} $
    * input signal $\\hat c[\\cdot]$:
        * length: $ N_s = {len(c_in1)}$
    * (RRC) pulse filter $\\psi(\\xi)$:
        * roll-off factor $\\beta = {bet}$
        * symbol period $T_s = {Ts}$
        * sampling rate $f_s = N_{{os}}/T_s = {fs}$
        * length: $N_{{\\psi}} = N_s = {Ns} $
    * length $u(\\xi): N_u = N_s = {Ns} $


"""))
zm = range(Ns//2, Ns//2 + Nos*3)
Visualizer.twin_zoom_plot(r'$|\psi(\xi)|^2$', np.abs(psi_xi) ** 2, zm, h_ind, xlabel=r'$\xi$')
Visualizer.plot_amp_and_phase(xi,u_in,r'$\xi$',r'u(\xi)')

In [ ]:
# 4) pre-equalize
# normalizing
u1 = mu*u_in

# #scaling
new_amp = np.sqrt(1-np.exp(-np.abs(u1)**2))
b_in1 = new_amp*np.exp(1j*np.angle(u1))


# Pre-compensation
b_in2 = b_in1*np.exp(-1j*xi**2*(L/Zn))

# re normalize in case of spilling above 1
if np.max(np.abs(b_in2))> 1:
    b_in3 = b_in2 / np.max(np.abs(b_in2))
else:
    b_in3 = b_in2

# amp exactly 1 is also not great, lets shrink by epsilon if needed
if np.max(np.abs(b_in3))== 1:
    b_in = b_in3 * (1-1e-6)
else:
    b_in = b_in3

assert np.allclose(b_in, b_in2), "b_in and b_in2 are not close"

assert np.all(0 <= np.abs(b_in)) and np.all(np.abs(b_in) <= 1), 'scaled signal is not in the range [0,1]'

# plot
display(Markdown(f"""
## 4) Pre-equalize
* normalizing: $u_1(\\xi)= \\mu \\cdot u(\\xi) $
* scaling: $b_1(\\xi) = \\sqrt{{1 - e^{{-|u_1 (\\xi)|^2}} }} \\cdot e^{{j\\angle u_1 (\\xi)}}$
* pre-compensation: $b(\\xi) = b_1(\\xi) \\cdot e^{{-j\\xi^2L/Z_n}}$
"""))

Visualizer.plot_amp_and_phase(xi,u1, r'$\xi$', r'u_1(\xi)')
Visualizer.plot_amp_and_phase(xi,b_in1, r'$\xi$', r'b_1(\xi)')
Visualizer.plot_amp_and_phase(xi,b_in, r'$\xi$', r'b(\xi)')


In [ ]:
np.max(np.abs(b_in))

In [ ]:
# test - reverse scaling


# post compensation - it must be without ssf since we haven't reached there yet
b_out = b_in * np.exp(1j * xi**2 * (L/Zn))


# descale
angl = np.exp(1j * np.angle(b_out))
amp = np.abs(b_out)**2
delta = np.abs(1 - amp)
minlog = -np.log(delta)
u1_out_test = np.sqrt(minlog) * angl

# normalize
u_out_test = u1_out_test / mu

# assert np.allclose(u_out_test, u_in), 'test failed'

# plot
allclose = np.allclose(u_out_test, u_in)
result = '[V]' if allclose else '[X] <- failed'

display(Markdown(f"""
## Test 4) descaling and denormalizing {result}

* post compensation: $\\hat b_1 = b \\cdot e^{{i \\xi^2 L/Z_n}}$
* descaling: $\\hat u_1 = \\sqrt{{-\\log(1-|\\hat b_1|^2)}} \\cdot e^{{i \\angle \\hat b_1}}$
* denormalizing: $\\hat u(\\xi) = \\hat u_1(\\xi)/\\mu$
"""))

Visualizer.compare_amp_and_phase(xi, u_out_test, u_in, r'$\xi$', r'u(\xi)', "")
if not allclose:
    Visualizer.double_plot("delta", np.abs(u_out_test) - np.abs(u_in),np.angle(u_out_test) - np.angle(u_in),xi,xi,
                        r'$\Delta |u(\xi)|$', r'$\Delta \angle u(\xi)$',
                        xlabel1=r'$\xi$', xlabel2=r'$\xi$')

In [ ]:
## 4.5) zero padding
pad_l = ((Nnft-Ns))//2-1
pad_r = ((Nnft-Ns))//2+1
b_in_padded = np.pad(b_in, (pad_l, pad_r), mode='constant', constant_values=0)
display(Markdown(f"""
## 4.5) zero padding
extend $b(\\xi)$ from length $N_s$ to length of $N_{{NFT}}$

$ b_p = \\underbrace{{[0,0,...,0]}}_{{(N_{{NFT}}-N_s)/2-1}} \\oplus b(\\xi) \\oplus \\underbrace{{[0,0,...,0]}}_{{(N_{{NFT}}-N_s)/2+1}}$

where
* length of $b(\\xi)$: $ N_s = {Ns}$
* length of $b_p(\\xi)$: $N_{{NFT}} = {Nnft}$
"""))
Visualizer.plot_amp_and_phase(xi_padded,b_in_padded, r'$\xi$', r'b_p(\xi)')

In [ ]:
# INFT2 (params from matlab)

contspec = b_in_padded
bound_states = []
discspec = []
cst = 1  # continuous spectrum type - default is None (1 = b of xi)
dst = 0  # default is None


res = nsev_inverse(xi_padded, t_padded, contspec, bound_states, discspec,
                   cst=cst, dst=dst)
q_in = res['q']


# plot
display(Markdown(f"""
## 5) INFT
$q(t) = \\mathcal F^{{-1}}\\{{b_p(\\xi)\\}}$                 
$ \\qquad  N_t = D = \\lceil N_\\xi \\rceil = {Nnft} $
"""))
zm = np.where(np.abs(t_padded) < max(t_padded)/5)[0]
Visualizer.twin_zoom_plot('|q(t)|', np.abs(q_in), zm, t_padded, 't')

In [ ]:
# 5.5) cropping to center
qb = q_in[(Nnft-Nb)//2:(Nnft+Nb)//2]

display(Markdown(f"""
## 5.5) cropping to center
we will keep $N_b$ samples from the center of $q(t)$
                 
$ q_b(t) = q[\\frac{{N_{{NFT}}-N_b}}{{2}} : \\frac{{N_{{NFT}}+N_b}}{{2}}] $
* $ N_b = T_b/\\Delta t = {Nb} $

* $ \\Delta t = {dt:.2f} $
"""))
zm = np.where(np.abs(t) < max(t)/5)[0]
Visualizer.twin_zoom_plot('|q_b(t)|', np.abs(qb), zm, t, 't [ps]')



In [ ]:
# converting to the r.w.u (physical units)
q_p = qb*np.sqrt(Pn)

Pave = np.sum(np.abs(q_p)**2)*dt/Tb
average_power = 30+10*np.log10(Pave) # dbm

display(Markdown(f"""
## 5.6) converting to the r.w.u
$ q_{{physical}} = q_b(t) \\cdot \\sqrt{{P_n}} $
* $ P_n = {Pn:.2e} $
* dBm power: $ 30 + 10\log_{{10}} |(q_p)|^2 $
* average power: $ P_{{avg}} = \\frac{{1}}{{T_b}} \\int_{{-T_b/2}}^{{T_b/2}} |q_p(t)|^2 dt = {average_power:.2f}$ [dBm]
"""))
# Visualizer.my_plot(t, np.abs(q_p)**2, name=r'$|q_p(t)|^2$ [W]', xlabel='t [ps]')



dbm_power = 30+10 * np.log10(np.abs(q_p)**2)
Visualizer.my_plot(t, dbm_power, name=r'$|q_p(t)|^2$', xlabel='t [ps]', ylabel='[dBm]')



In [ ]:
if with_ssf:
    # preparation
    h = 6.62607015e-34  # planck constant [J*s]
    lambda_0 = 1.55e-6  # wavelength [m]
    C = 299792458  # speed of light [m/s]
    K_T = 1.13   # [unitless]
    # X_dBkm = 0.2  # fiber loss coefficient [dB/km]
    # X_km = 10 ** (X_dBkm / 10) # [1/km]
    X_km = 0.0461 # [1/km]
    # X = X_km * 1e-3 [1km/1000m]  #  [1/m]
    nu_0 = C / lambda_0  # frequency [Hz]

    D_Jm = 0.5 * h * nu_0 * K_T * X_km  # [J/m=Ws/m]
    # normalizer = 1e12 # ps->s and m->km
    D_unit_normalizer = 1e12
    # unexplained_normalizer = 1e-3
    D = D_Jm * D_unit_normalizer # D [v*ps/km]

    noise_amplitude = np.sqrt(D * dz / dt)

    N = int(L / dz)

    # start of the algorithm
    u = q_p
    Nt = np.max(u.shape)  # length
    w = 2.0 * np.pi / (Nt * dt) * np.fft.fftshift(np.arange(-Nt / 2.0, Nt / 2.0))

    # Half-step linear propagation
    half_step = np.exp((1j * beta2 / 2.0 * w ** 2) * dz / 2.0)
    full_step = np.exp((1j * beta2 / 2.0 * w ** 2) * dz)

    u = np.fft.ifft(np.fft.fft(u) * half_step) # half linear step -> [convolve with half_step]
    # print(f'looping {N} times')
    # print(f'noise amplitude: {noise_amplitude:.2e}')
    for i in range(N-1):
        # noise preparation
        noise = noise_amplitude * (np.random.randn(Nt) + 1j * np.random.randn(Nt))
        # if i % 500 == 0:
        #     avg_noise_power = 10*np.log10(np.sum(np.abs(noise)**2)*dt/Tb/1e-3) # dbm
        #     avg_signal_power = 10*np.log10(np.sum(np.abs(u)**2)*dt/Tb/1e-3) # dbm
        #     snr_db = avg_signal_power - avg_noise_power
        #     print(f'{i:04d}) noise_power: {avg_noise_power:.2f}[dBm], signal_power: {avg_signal_power:.2f}[dBm], snr: {snr_db:.2f} [dB]')
            # Visualizer.compare_amp_and_phase_log(t, u, q_p, r'$t$', r'q_z(t)','before and after SSF')
        # check if u became nan
        if np.any(np.isnan(u)):
            print(f'nan at iteration {i}')
            break


        u *= np.exp(1j * gamma_eff * dz * np.abs(u) ** 2)   # Nonlinear propagation
        u += noise                                          # Noise addition
        u = np.fft.ifft(full_step * np.fft.fft(u))          # full linear step

    u *= np.exp(1j * gamma_eff * dz * np.abs(u) ** 2)           # Nonlinear propagation

    # noise addition
    noise = noise_amplitude * (np.random.randn(Nt) +1j * np.random.randn(Nt))
    u += noise
    u = np.fft.ifft(np.fft.fft(u) * half_step) # half linear step

    qz = u
else:
    qz = q_p


display(Markdown(f"""
## 6) Split Step Fourier
$ q_p(t) \\rightarrow \\boxed{{SSF}} \\rightarrow q_z(t) $
### params:
* $ \\beta_2 = {beta2} [ps^2/km] $
* $ \\gamma = \\gamma_{{eff}}= {gamma_eff:.2f} [1/km \\cdot W] $
* $ T_0 = {T0} [ps] $
* $ \\Delta t = {dt:.2f} [ps] $
* $ \\sout{{Z_n = La = {La} [km]}} $
* $ L = {L} [km] $
* $ \\Delta z = {dz} [km] $
* $ N_t = {Nt} $

### algorithm
#### setting up variables (done once before)
* input: $q_p\\in \\mathbb{{C}}^{{N_t}}$
* calculating D:
    * $ D = 0.5h \\nu_0 K_T \\chi =$ {D_Jm:.2e} $[J/km=W \\cdot s/km] (\cdot $ {D_unit_normalizer:.0e} $) \\rightarrow $ {D:.2e} $ [W \\cdot ps/km] $
        * $ h = $ {h:.2e} $ [J \\cdot s] $
        * $ K_T = {K_T} [\cdot] $
        * $ C = {C} [m/s] $
        * $ \\lambda_0 =$ {lambda_0} $[m] $
        * $ \\nu_0 = C / \\lambda_0 =$ {nu_0:.2e} $ [Hz] $
        * $ \\chi = $ {X_km:.3f} [1/km]
* noise amplitude: $\\sqrt{{\\frac{{D\\cdot \\Delta_z}}{{dt}}}}=$ {noise_amplitude:.2e} $ [\sqrt{{W}}]$
* generate N noise vectors: $ n_i \\in \\mathbb{{C}}^{{N_t}} \\sim N(0,1) \\forall i\\in [0:N-1] $
* angular frequency array: $w = [0,...,\\frac{{N_t}}{{2}}-1,\\frac{{-N_t}}{{2}},...,-1] \\cdot \\frac{{2 \\pi}}{{N_t \\cdot dt}}$
* $ v_{{half}} = \\exp(\\frac{{j \\cdot \\beta_2}}{{2}} \\cdot w^2 \\cdot \\frac{{\\Delta_z}}{{2}}) $
* $ v_{{full}} = \\exp(\\frac{{j \\cdot \\beta_2}}{{2}} \\cdot w^2 \\cdot \\Delta_z) $
#### execution (per input q):
* half linear step: $ U = q_p(t) \\circledast v_{{half}} $
* Loop $N=\\frac{{L}}{{\\Delta_z}}-1$ times:
    1. nonlinear step: $U = U \\cdot \\exp(j \\gamma |U|^2 \\Delta_z)$
    2. add noise: $U = U + \\sqrt{{\\frac{{D\\cdot \\Delta_z}}{{dt}}}}\\cdot n_i$ 
    3. full linear step:  $U = U \\circledast v_{{full}}$
* nonlinear step: $U = U \\cdot \\exp(j \\gamma |U|^2 \\Delta_z)$
* add noise: $U = U + \\sqrt{{\\frac{{D\\cdot \\Delta_z}}{{dt}}}}\\cdot n_{{N-1}}$
* half linear step: $ q_z = U \\circledast v_{{half}}$
"""))

Visualizer.compare_amp_and_phase_log(t, qz, q_p, r'$t$', r'q_z(t)','before and after SSF')

In [ ]:
# 6.5) descaling and padding:
q_s = qz/np.sqrt(Pn) # rescale to soliton units

assert Nnft > Nb, 'Nnft must be larger than Nb'

q_pad = np.pad(q_s, (((Nnft-Nb))//2, ((Nnft-Nb))//2), mode='constant', constant_values=0) # pad to length of N_NFT
    
display(Markdown(f"""
## 6.5) descaling and padding:
* rescale the pulse to soliton units 
    * $ q_s(t) = q_z(t) / \\sqrt{{P_n}} $
        * $ P_n =$ {Pn:.2e} $ [W] $
* pad the pulse to length of $N_{{NFT}}$
    * $ q_{{pad}}(t) = [0,0,...,0] \\oplus q_s(t) \\oplus [0,0,...,0] $
        * $ N_{{pad}} = N_{{NFT}} = {Nnft} $
        * $ t_{{padded}} \\in [\\frac{{-N_{{NFT}}}}{{2}}, \\frac{{N_{{NFT}}}}{{2}}] \\cdot \\Delta t = [{-Nnft//2*dt:.2f}, {Nnft//2*dt:.2f}] $
"""))

Visualizer.plot_amp_and_phase(t_padded, q_pad, r'$t_{padded}$', r'q_p(\xi)')

In [ ]:
cst = 1 # type of continuous spectrum: 1 = a and b
dst = 0 # type of discrete spectrum: 0 = none
res = nsev(q_pad, t_padded, Xi1=XI[0], Xi2=XI[1],
           M=Nnft, display_c_msg=True, cst=cst)
assert res['return_value'] == 0, "NFT failed"
b_out_padded = res['cont_b']

# x7 = x7 * np.exp(-1j * xivec**2 * (L))

display(Markdown(f"""
## 7) NFT
Forward NFT back to $\\xi$ domain  
$$ \\hat b_{{pad}}(\\xi) = \\mathcal F \\{{ q_{{pad}}(t) \\}} $$
"""))

Visualizer.compare_amp_and_phase(xi_padded, b_out_padded, b_in_padded, r'$\xi$', r'b_p(\xi)',"")

In [ ]:
## 7.5 cropping to center
b_out = b_out_padded[(Nnft-Ns)//2-1:(Nnft+Ns)//2-1]

display(Markdown(f"""
## 7.5) cropping to center
we will keep $N_s$ samples from the center of $\\hat b_{{pad}}(\\xi)$

$ \\hat b(\\xi) = \\hat b_{{pad}}[\\frac{{N_{{NFT}}-N_s}}{{2}} : \\frac{{N_{{NFT}}+N_s}}{{2}}] $
* $ N_s = {Ns} $
"""))
# Visualizer.plot_amp_and_phase(xi, b_out, r'$\xi$', r'b(\xi)')
Visualizer.compare_amp_and_phase(xi, b_out, b_in, r'$\xi$', r'b(\xi)',"", square=False)

In [ ]:
# with_ssf = False

In [ ]:
# 8) Equalize
# post compensation
if with_ssf:
    b_out1 = b_out * np.exp(-1j * xi**2 * (L/Zn))
else:
    b_out1 = b_out * np.exp(1j * xi**2 * (L/Zn))

# descale
u1_out = np.sqrt(-np.log(1 - np.abs(b_out1)**2)) * np.exp(1j * np.angle(b_out1))

# de-normalize
u_out = u1_out / mu 

display(Markdown(f"""
## 8) Equalize

* post compensation: 
    * with ssf:
        * $\\hat b_1(\\xi) = \\hat b(\\xi) \\cdot e^{{-i \\xi^2 L/Z_n}}$
    * without ssf:
        * $\\hat b_1(\\xi) = \\hat b(\\xi) \\cdot e^{{i \\xi^2 L/Z_n}}$

* descaling: $\\hat u_1(\\xi) = \\sqrt{{-\\log(1-|\\hat b(\\xi)|^2)}} \\cdot e^{{i \\angle \\hat b(\\xi)}}$
* denormalizing: $\\hat u(\\xi) = \\hat u_1(\\xi)/\\mu$
"""))

Visualizer.compare_amp_and_phase(xi,u_out, u_in, r'$\xi$', r'u(\xi)', "", square=False)

In [ ]:
## 8.7) replace nan with previous neighbors
# for each sample that is nan, we'll replace it with the previous sample
# this is done in order to avoid nan in the fft
u_out_no_nan = np.copy(u_out)
nan_mask = np.isnan(u_out_no_nan)
nan_indices = np.where(nan_mask)[0]
for i in nan_indices:
    u_out_no_nan[i] = u_out_no_nan[i-1]

# plot
display(Markdown(f"""
## 8.7) replace nan with previous neighbors
we will replace each nan with the previous sample
"""))
Visualizer.compare_amp_and_phase(xi,u_out_no_nan, u_in, r'$\xi$', r'u(\xi)', "", square=False)


In [ ]:
## 9) match filter

# convolve with RRC
c_out1 = np.fft.ifft(np.fft.fft(u_out_no_nan)* psi_t) / Nos


# downsample
start = 0
stop = Ns  # + over_sampling
step = Nos
c_out = c_out1[start:stop:step] 

display(Markdown(f"""
## 9) Match Filter
convolve again with the rrc filter and sample every $ N_{{os}}={Nos} $ points, those points are free from inter symbol interference (aliasing)  
* $ \\hat{{c}}_1 = \\hat{{u}}(\\xi) \\ast \\psi(\\xi) = \\mathcal F^{{-1}}\{{U \\cdot \\Psi \}} $
* $ \\hat{{c}}[n] = \\hat{{c}}_1[nN_{{os}}] / N_{{os}} $
    * $ N_{{os}} = {Nos} $
"""))


Visualizer.compare_amp_and_phase(xi, c_out1,c_in1, r'$\xi$', r'\hat c_1(\xi)', "",square=False)
Visualizer.compare_amp_and_phase(range(len(c_in)), c_out,c_in, r'$n$', r'\hat c[n]', "downsampled",square=False)
Visualizer.twin_zoom_plot('downsampled c[n]',np.real(c_out),range(0,50),function='stem')
Visualizer.plot_constellation_map_with_points(c_out, M_QAM, 'after depulse shaping')
Visualizer.eye_diagram(c_out, sps=sps)

In [ ]:
s_out = modem.demodulate(c_out)

#plot
display(Markdown(f"""
## 10) Demodulation
$ \\hat{{s}}_b = f_{{DEMOD}}(\\hat{{c}}) $
"""))
Visualizer.print_bits(s_out, sps, 'message after channel')

In [ ]:
num_errors = (message_s_bin != s_out).sum()
N_bin = len(message_s_bin)
ber = num_errors / N_bin

display(Markdown(f"""
## Evaluate
calculate ber:
"""))
print(f'ber = {ber} = {num_errors}/{N_bin}')
